Import Libraries


In [1]:
# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

Load PDF

In [5]:
# Load PDF
local_path = "metagpt.pdf"
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
    print(f"PDF loaded successfully: {local_path}")
else:
    print("Upload a PDF file")

PDF loaded successfully: metagpt.pdf


Split Texts

In [6]:
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)
print(f"Text split into {len(chunks)} chunks")

Text split into 109 chunks


Create vector database


In [8]:
# Create vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="gemma2:2b"),
    collection_name="local-rag"
)
print("Vector database created successfully")

Vector database created successfully


Set up LLM and Retrieval


In [10]:
# Set up LLM and retrieval
local_model = "gemma2:2b"  # or whichever model you prefer
llm = ChatOllama(model=local_model)

In [11]:
# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate 2
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

Create Chain

In [12]:
# RAG prompt template
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)


In [13]:
# Create chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Chat with PDF


In [14]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

In [15]:
# Example 1
chat_with_pdf("What is the main idea of this document?")

This document explores and details **MetaGPT**, a novel approach for software development using large language models (LLMs) and specialized agents. 

Here's a breakdown of its key points:

* **The Problem:** Existing LLMs struggle with complex software engineering tasks requiring detailed system design, requirements analysis, and code generation.
* **The Solution: MetaGPT**  utilizes a structured approach to address these challenges by:
    * **Specialized Agents:** Dividing tasks into roles specific agents specialize in (e.g., requirement analysis, code generation) for efficiency.
    * **Shared Message Pool:** Enabling efficient communication and information sharing between agents to streamline workflow.
    * **Structured Output Formats:**  Enabling the generation of detailed reports and design plans that can be easily interpreted and implemented.

* **Advantages of MetaGPT over other LLMs:**  MetaGPT excels in handling complex software development through:
     * **Automated Requirement Analysis:** Translating user needs into detailed system designs.
     * **Improved Communication Efficiency:** Eliminating communication bottlenecks using a centralized message pool. 


Ultimately, the document advocates for MetaGPT as a promising methodology to unlock the potential of LLMs in complex software engineering tasks. 


In [17]:
# Example 2
chat_with_pdf("Can you explain why is MetaGPT the solution?")

MetaGPT, as described in this document, offers a unique approach to software development that addresses several limitations of existing solutions. 

Here's a breakdown of its strengths and how it positions itself as a potential solution:

**1. Addressing the Challenges:**

* **Ambiguity & Information Validity:** MetaGPT tackles two sub-challenges:
    * **Unfolding Natural Language:** It aims to accurately translate short descriptions into clear instructions for execution, reducing ambiguity and improving accuracy. 
    * **Maintaining Contextual Validity:**  It handles complex tasks within lengthy contexts (like large software development) while ensuring information relevance, focusing on the relevant data without getting bogged down.

**2. Unique Features for Complex System Development:**

* **Systematic Decomposition:** MetaGPT tackles the complexities of developing systems by breaking down requirements into manageable steps. This is achieved through SOPs (Standard Operating Procedures). 
* **Structured Messaging & Feedback:**  It provides a structured platform for communication between developers and ensures clear feedback mechanisms, reducing errors and improving efficiency.
* **Collaboration Focus:** It emphasizes collaboration with skilled development teams to address challenges that individual AI models might struggle with.

**3. Why MetaGPT is Superior to the Alternatives:**

* **Auto-GPT (Torantulino et al., 2023), Langchain (Chase, 2022), and Agent-Verse (Chen et al., 2023):** These models demonstrate strong problem-solving abilities. However, they still face challenges in breaking down requirements into actionable steps and handling complex contexts efficiently. 
* **Chat-Dev (Zhao et al., 2023):** While potentially useful for specific tasks, it lacks the system-wide focus and structured framework needed to handle complex software development.

**In Summary:**

MetaGPT offers a distinct approach by combining specialized structures and communication mechanisms within an SOP framework.  This allows it to deliver on complex software development needs while addressing key challenges that other approaches haven't fully tackled, particularly in terms of system architecture and complexity.


 **Note**: The provided text is quite technical and provides detailed explanations about the workings of MetaGPT. If you need a more simplified explanation or have specific questions about its features and how it works, feel free to ask! 


In [18]:
# Example 2
chat_with_pdf("Summarize only the Experiment and Results part of the paper.")

This text describes a study on using large language models (LLMs) for developing complex software systems.  Here's a summary of their experiment and results:

**Experiment:** The researchers designed an LLM-driven system called "MetaGPT" to help users develop programs in natural language. They compare MetaGPT with other state-of-the-art LLMs like AutoGPT, Langchain, and Agent-Verse.  To evaluate its effectiveness, they use several testing methods:

* **Requirements Breakdown:** MetaGPT breaks down complex programming requirements into concrete designs and plans for execution.
* **Real-time Visualization:** The system allows for real-time visualization of code and user actions for better understanding and feedback. 
* **Accuracy Tests:**  The code developed by MetaGPT is tested rigorously for accuracy using Python's unittest library.

**Results:** 
* **Efficiency & Accuracy:** MetaGPT demonstrated high efficiency and accuracy in creating working software, outperforming the traditional methods of programming. This suggests a significant step towards automating complex development tasks. 
* **User-Friendly Experience:** The use of natural language allows anyone to participate, making it a more accessible tool for programmers.

**Key Takeaways:** 
The paper highlights that MetaGPT has significant potential in revolutionizing software development. It offers users a faster, more user-friendly way to build complex systems, while also demonstrating greater accuracy and efficiency compared to traditional methods.


Let me know if you'd like me to elaborate on specific parts of the experiment or results! 
